In [3]:
import numpy as np
import pandas as pd

graph = np.array(
    [
        [0,8,15,10,21,26],
        [8,0,12,7,16,23],
        [15,12,0,19,9,11],
        [10,7,19,0,11,17],
        [21,16,9,11,0,13],
        [26,23,11,17,13,0]
    ]
)
demand = np.array([10,8,22,18,7,55])
distance = 10  #covering distance
P = 2   # number of facilities
alpha = 2  
inf = 1e9
iteration = 1
alpha_iteration = 0
upper_bound=[]
lower_bound=[]
zero_array = [0]*6
max_iteration = 4


a_ij = np.where(graph<=10,1,0)
X_list = np.array([1,1,0,0,0,0])
# print(a_ij)

# X_j column 
def find_X_j(series):

    arr1 = list(series)
    arr2 = np.zeros(6)
    while True:
        L = arr1.index(max(arr1))
        arr2[L]=1
        if arr2.sum()==2:
            break
        arr1[L]=-inf
      
    return arr2

#LB column
# X_j가 배정됨에 따라 커버되는 new_Zi 를 구하는 것은 그냥 a_ij의 i 번째 행을 갖다 쓰면 된다..
def find_LB(df):
    arr=[]
    for n,i in enumerate(df['X_j']):
        if i==1:
            sigma = np.dot(a_ij[n],df['h_i'])
            arr.append(sigma)
        elif i==0:
            arr.append(0)
    return arr
#find t_n, step size
def find_t_n(df):
    t_n = alpha*(UB-LB)/sum((df['sigma(aX)-Z']**2))
    return t_n

#display option
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

# Create Dataframe
index_list = ['A','B','C','D','E','F']
table = np.zeros((len(index_list),1))

df = pd.DataFrame(table)
df.index = index_list

#초기 Dataframe 생성
df['h_i'] = demand
h_bar = df['h_i'].mean()
df['lambda_i']=np.array([h_bar+0.5*(i-h_bar) for i in df['h_i']])
df['Z_i']= np.where(df['h_i']-df['lambda_i']>=0,1,0)
df['(h_i-lambda_i)*Z_i'] = (df['h_i']-df['lambda_i'])*df['Z_i']
df['sigma(a*lambda)'] = np.dot(a_ij,df['lambda_i'])
df['X_j'] = find_X_j(df['sigma(a*lambda)'])
df['sigma(a*lambda)X_j'] = df['sigma(a*lambda)']*df['X_j']
df['sigma(aX)']= np.dot(a_ij,df['X_j'])
df['sigma(aX)-Z']=df['sigma(aX)']-df['Z_i']
df['LB']=find_LB(df)
df['zero_array']=zero_array

upper_bound.append((df['(h_i-lambda_i)*Z_i']+df['sigma(a*lambda)X_j']).sum())
lower_bound.append(df['LB'].max())
UB = min(upper_bound)
LB = max(lower_bound)
t_n = find_t_n(df)          #1st t_n value
print("초기값입니다.")
print("LB:{0},UB:{1:.3f},alpha:{2:.3f},t_n:{3:.3f}".format(LB,UB,alpha,t_n))
# print(df)
#1st iteration 종료 -------
# print("초기값 df:", df)




#iteration을 통한 새로운 df 생성. -------
#lambda_i+1
def find_next_sheet(df):
    s1=df['zero_array']
    s2=df['lambda_i']-t_n*df['sigma(aX)-Z']
    s3=pd.concat([s1, s2],ignore_index=True, axis=1).max(axis=1)
    print(s1,s2,s3)
    df['lambda_i']=s3
    df['Z_i']= np.where(df['h_i']-df['lambda_i']>=0,1,0)
    df['(h_i-lambda_i)*Z_i'] = (df['h_i']-df['lambda_i'])*df['Z_i']
    df['sigma(a*lambda)'] = np.dot(a_ij,df['lambda_i'])
    df['X_j'] = find_X_j(df['sigma(a*lambda)'])
    df['sigma(a*lambda)X_j'] = df['sigma(a*lambda)']*df['X_j']
    df['sigma(aX)']= np.dot(a_ij,df['X_j'])
    df['sigma(aX)-Z']=df['sigma(aX)']-df['Z_i']
    df['LB']=find_LB(df)
    return df

def find_next_bound(df):
    upper_bound.append((df['(h_i-lambda_i)*Z_i']+df['sigma(a*lambda)X_j']).sum())
    lower_bound.append(df['LB'].unique().sum())
    UB = min(upper_bound)
    LB = max(lower_bound)
    return UB,LB

# print(upper_bound,lower_bound)
# print(t_n)

# iteration
while abs(LB-UB)>0.1 and alpha > 0.1 and alpha_iteration<max_iteration:
    alpha_iteration +=1  # 요게 max가 되면 계산 종료
    iteration +=1
    df=find_next_sheet(df)  # df 갱신
    UB_new, LB_new = find_next_bound(df) # bound 갱신
    # while True:     # iteration을 계속 진행할지, 아니면 alpha를 바꿀지를 결정하는 단계
    if UB_new < UB :  # 좋아지면?
        alpha_iteration = 0 # 알파 카운팅횟수 초기화
        t_n = find_t_n(df)
        UB,LB  = UB_new,LB_new     # UB, LB 값 더 좋아졌으니 Best 값 갱신
        print("{0}번째 반복입니다. UB가 좋아졌네요!".format(iteration))
    elif UB_new >= UB: # 안좋아지면?
        alpha =1
        t_n = find_t_n(df)
        print("{0}번째 반복입니다. UB가 좋아지지 않아 alpha를 갱신합니다.".format(iteration))
    
    print("LB:{0},UB:{1:.3f},alpha:{2:.3f},t_n:{3:.3f}".format(LB,UB,alpha,t_n))
    print(df)
    print("Upper Bound array :{}".format(upper_bound))
    print("---------------------------------------------------")


df

초기값입니다.
LB:36,UB:114.500,alpha:2.000,t_n:11.214
A    0
B    0
C    0
D    0
E    0
F    0
Name: zero_array, dtype: int64 A    -7.428571
B    -8.428571
C    32.214286
D    -3.428571
E    13.500000
F    48.714286
dtype: float64 A     0.000000
B     0.000000
C    32.214286
D     0.000000
E    13.500000
F    48.714286
dtype: float64
2번째 반복입니다. UB가 좋아지지 않아 alpha를 갱신합니다.
LB:36,UB:114.500,alpha:1.000,t_n:15.700
     0  h_i   lambda_i  Z_i  (h_i-lambda_i)*Z_i  sigma(a*lambda)  X_j  \
A  0.0   10   0.000000    1           10.000000         0.000000  0.0   
B  0.0    8   0.000000    1            8.000000         0.000000  0.0   
C  0.0   22  32.214286    0           -0.000000        45.714286  1.0   
D  0.0   18   0.000000    1           18.000000         0.000000  0.0   
E  0.0    7  13.500000    0           -0.000000        45.714286  0.0   
F  0.0   55  48.714286    1            6.285714        48.714286  1.0   

   sigma(a*lambda)X_j  sigma(aX)  sigma(aX)-Z  LB  zero_array  
A            0.0

,0,h_i,lambda_i,Z_i,(h_i-lambda_i)*Z_i,sigma(a*lambda),X_j,sigma(a*lambda)X_j,sigma(aX),sigma(aX)-Z,LB,zero_array
A,0.0,10,7.918320,1,2.081680,17.213294,0.0,0.000000,0.0,-1.0,0,0
B,0.0,8,7.918320,1,0.081680,17.213294,0.0,0.000000,0.0,-1.0,0,0
C,0.0,22,20.853307,1,1.146693,50.808333,1.0,50.808333,2.0,1.0,29,0
D,0.0,18,1.376653,1,16.623347,17.213294,0.0,0.000000,0.0,-1.0,0,0
E,0.0,7,29.955026,0,-0.000000,50.808333,1.0,50.808333,2.0,2.0,29,0
F,0.0,55,30.465939,1,24.534061,30.465939,0.0,0.000000,0.0,-1.0,0,0


In [93]:
df['X_j'].unique().sum()

1.0